In [1]:
from pathlib import Path
wd = Path.cwd()
wd = wd.parent.parent
wd = wd / 'merged_data' 
sub1 = str(wd) + '/subtask1.parquet'
print(sub1)

/home/matijak/Documents/programiranje/projects/semeval/merged_data/subtask1.parquet


In [2]:
import pandas as pd
df = pd.read_parquet(sub1)
#df.head(n=20)
df.aggregate

<bound method DataFrame.aggregate of      lang    art_name             entity start   end       class1  \
0      BG  BG_670.txt              Запад   152   156   Antagonist   
1      BG  BG_670.txt                САЩ   530   532   Antagonist   
2      BG  BG_670.txt               НАТО   535   538   Antagonist   
3      BG  BG_670.txt            Украйна   578   584   Antagonist   
4      BG  BG_670.txt  украински войници   633   649     Innocent   
...   ...         ...                ...   ...   ...          ...   
2530   PT  PT_159.txt              China   371   375     Innocent   
2531   PT   PT_74.txt           Pokrovsk  1139  1146     Innocent   
2532   PT   PT_31.txt            Ucrânia   313   319   Antagonist   
2533   PT   PT_31.txt                EUA   327   329   Antagonist   
2534   PT   PT_31.txt            Moscovo  1999  2005  Protagonist   

                                          classes2  \
0     [Conspirator, Instigator, Foreign Adversary]   
1                         

In [3]:
def labelNum(row):
    if row['class1'] == 'Antagonist':
        return int(0)
    if row['class1'] == 'Innocent':
        return int(1)
    if row['class1'] == 'Protagonist':
        return int(2)

def textCon(row):
    text1 = "Consider the role of " + row['entity'] + " in the text."
    text = text1 + " " + row['text']
    return text

df['label'] = df.apply(labelNum,axis=1)
#print(df['label'].value_counts(),
#df['class1'].value_counts())
df['input'] = df.apply(textCon,axis=1)
#print(df['input'].iloc[0])

In [4]:
data = df.loc[ : , ['input','label']]
data.aggregate

<bound method DataFrame.aggregate of                                                   input  label
0     Consider the role of Запад in the text. Опитът...      0
1     Consider the role of САЩ in the text. Опитът н...      0
2     Consider the role of НАТО in the text. Опитът ...      0
3     Consider the role of Украйна in the text. Опит...      0
4     Consider the role of украински войници in the ...      1
...                                                 ...    ...
2530  Consider the role of China in the text. A tran...      1
2531  Consider the role of Pokrovsk in the text. Rús...      1
2532  Consider the role of Ucrânia in the text. Quai...      0
2533  Consider the role of EUA in the text. Quais fo...      0
2534  Consider the role of Moscovo in the text. Quai...      2

[2535 rows x 2 columns]>

In [5]:
from sklearn.model_selection import train_test_split
X = data["input"]
y = data["label"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train.head(), X_test.head(), y_train.head(), y_test.head()


(1979    Consider the role of Europa in the text. O pas...
 1110    Consider the role of पश्चिमी देश in the text. ...
 157     Consider the role of Русия in the text. Столте...
 1210    Consider the role of ट्रंप in the text. 'रूस-य...
 888     Consider the role of मोदी in the text. नई दिल्...
 Name: input, dtype: object,
 1414    Consider the role of रूस in the text. प्रधानमं...
 1776    Consider the role of ओरबान in the text. Viktor...
 2323    Consider the role of Comissão Europeia in the ...
 1389    Consider the role of रुबिज़ने in the text. यूक...
 1379    Consider the role of रूस in the text. रूस-यूक्...
 Name: input, dtype: object,
 1979    1
 1110    0
 157     0
 1210    2
 888     2
 Name: label, dtype: int64,
 1414    2
 1776    0
 2323    2
 1389    1
 1379    2
 Name: label, dtype: int64)

In [6]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [7]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [8]:
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer

model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3).to(device)
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from datasets import Dataset

# Convert pandas DataFrame/Series to Hugging Face Dataset
train_data = pd.DataFrame({'text': X_train, 'label': y_train})
test_data = pd.DataFrame({'text': X_test, 'label': y_test})

train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [10]:
def preprocess_function(examples):
    return tokenizer(examples['text'], padding=True, truncation=True, max_length=128)

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2028 [00:00<?, ? examples/s]

Map:   0%|          | 0/507 [00:00<?, ? examples/s]

In [11]:
train_dataset[0]

{'text': 'Consider the role of Europa in the text. O passado domingo foi o dia mais quente já registado no mundo, diz observatório europeu\n\nO passado domingo, 21 de julho, foi o dia mais quente de que há registo a nível global, de acordo com os dados preliminares do Serviço de Alterações Climáticas Copernicus, da União Europeia.\n\nA temperatura média do ar à superfície do planeta no domingo atingiu os 17,09°C —um pouco acima do recorde anterior, de 6 de julho do ano passado, de 17,08°C.\n\nAs ondas de calor atingiram grandes áreas dos Estados Unidos, Europa e Rússia na última semana.\n\nO observatório Copernicus confirmou à agência Reuters que o recorde de temperatura média diária estabelecido no ano passado parece ter sido batido no domingo, de acordo com os seus registos, que remontam a 1940.\n\nSegundo o Copernicus, o novo recorde diário pode ser ultrapassado nos próximos dias. As temperaturas devem descer a seguir, mas são esperadas ainda flutuações nas próximas semanas.\n\n"O q

In [12]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
print(train_dataset)

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 2028
})


In [13]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64)
optimizer = AdamW(model.parameters(), lr=2e-5)

In [14]:
num_epochs = 3
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    # Initialize tqdm progress bar for training
    train_progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}/{num_epochs}")
    
    for batch in train_progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Calculate accuracy
        preds = torch.argmax(logits, dim=-1)
        #print(preds)
        correct_predictions += (preds == labels).sum().item()
        total_predictions += labels.size(0)
        
        # Update tqdm description with current loss
        train_progress_bar.set_postfix({'loss': loss.item()})
    
    avg_train_loss = total_loss / len(train_dataloader)
    train_accuracy = correct_predictions / total_predictions
    
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Training loss: {avg_train_loss:.4f}, Training accuracy: {train_accuracy:.4f}")
    
    # Test phase
    model.eval()
    test_loss = 0
    correct_test_predictions = 0
    total_test_predictions = 0
    
    # Initialize tqdm progress bar for test
    test_progress_bar = tqdm(test_dataloader, desc=f"Test Epoch {epoch + 1}/{num_epochs}")
    
    with torch.no_grad():
        for batch in test_progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits
            
            test_loss += loss.item()
            
            # Calculate test accuracy
            preds = torch.argmax(logits, dim=-1)
            correct_test_predictions += (preds == labels).sum().item()
            total_test_predictions += labels.size(0)
            
            # Update tqdm description with current loss
            test_progress_bar.set_postfix({'loss': loss.item()})
    
    avg_test_loss = test_loss / len(test_dataloader)
    test_accuracy = correct_test_predictions / total_test_predictions
    
    print(f"Test loss: {avg_test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")

Training Epoch 1/3: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:25<00:00,  1.24it/s, loss=1.1]


Epoch 1/3
Training loss: 1.0659, Training accuracy: 0.4882


Test Epoch 1/3: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  4.13it/s, loss=1.12]


Test loss: 1.0577, Test accuracy: 0.4458


Training Epoch 2/3: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:24<00:00,  1.29it/s, loss=1.01]


...

Test loss: 1.0245, Test accuracy: 0.5089


Training Epoch 3/3: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:24<00:00,  1.29it/s, loss=1.05]


Epoch 3/3
Training loss: 0.9780, Training accuracy: 0.5247


Test Epoch 3/3: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  4.03it/s, loss=1.21]

Test loss: 0.9834, Test accuracy: 0.5483
